**this notebook is an answer for the point number 6 which states:-**

**6. The embedding model we used here is a basic embedding model, change the model and use openai's embedding model 'text-embedding-ada-002'**

**Unfortunately i couldn't create another index, as i need to pay for creating another index, but the code is here**

**when i tried to make openai api key i get message states:- "Not available 
OpenAI's services are not available in your country."**

In [1]:
import openai

openai.api_key = "YOUR_API_KEY" # when i tried to make openai api key i get message states:- "Not available OpenAI's services are not available in your country."
# get API key from top-right dropdown on OpenAI website

openai.Engine.list()  # check we have authenticated

In [2]:
MODEL = "text-embedding-ada-002"

res = openai.Embedding.create(
    input=[
        "Sample document text goes here",
        "there will be several phrases in each batch"
    ], engine=MODEL
)

In [3]:
# extract embeddings to a list
embeds = [record['embedding'] for record in res['data']]

In [ ]:
import pinecone

# initialize connection to pinecone (get API key at app.pinecone.io)
pinecone.init(
    api_key="dae36959-08bc-4f9e-8468-3741321386b6",
    environment="us-east-1-aws"  # find next to API key in console
)

# check if 'openai' index already exists (only create index if not)
if 'openai' not in pinecone.list_indexes():
    pinecone.create_index('openai', dimension=len(embeds[0]))
# connect to index
index = pinecone.Index('openai')

In [ ]:
from datasets import load_dataset

# load the first 1K rows of the TREC dataset
trec = load_dataset('trec', split='train[:1000]')

In [ ]:
from tqdm.auto import tqdm  # this is our progress bar

batch_size = 32  # process everything in batches of 32
for i in tqdm(range(0, len(trec['text']), batch_size)):
    # set end position of batch
    i_end = min(i+batch_size, len(trec['text']))
    # get batch of lines and IDs
    lines_batch = trec['text'][i: i+batch_size]
    ids_batch = [str(n) for n in range(i, i_end)]
    # create embeddings
    res = openai.Embedding.create(input=lines_batch, engine=MODEL)
    embeds = [record['embedding'] for record in res['data']]
    # prep metadata and upsert batch
    meta = [{'text': line} for line in lines_batch]
    to_upsert = zip(ids_batch, embeds, meta)
    # upsert to Pinecone
    index.upsert(vectors=list(to_upsert))


In [ ]:
query = "What caused the 1929 Great Depression?"

xq = openai.Embedding.create(input=query, engine=MODEL)['data'][0]['embedding']

In [ ]:
res = index.query([xq], top_k=5, include_metadata=True)

In [ ]:
for match in res['matches']:
    print(f"{match['score']:.2f}: {match['metadata']['text']}")


**shortly https://docs.pinecone.io/docs/openai is the answer for this question**